In [0]:
sales_df = spark.read.option("header", "true").option("inferSchema", "true").csv("/Volumes/upload/default/files/sales_data.csv")
store_df = spark.read.option("header", "true").option("inferSchema", "true").csv("/Volumes/upload/default/files/store_data.csv")
display(sales_df.sample(fraction=0.01))
display(store_df.sample(fraction=0.4))

sale_id,store_id,product_id,sale_date,quantity,total_amount
sale_301,16.0,15,2022-12-17,47,399.21
sale_327,15.0,6,2022-04-16,-4,-50.0
sale_339,15.0,16,2021-10-05,55,1892.32
sale_396,6.0,50,2022-12-02,51,1555.74
sale_435,5.0,44,2022-10-13,86,2515.89
sale_472,9.0,49,2020-09-26,84,1178.18
sale_681,3.0,30,2022-03-04,94,2311.25
sale_782,7.0,34,2021-05-19,51,2133.53
sale_865,10.0,12,2021-07-01,39,1581.28
sale_871,16.0,44,2022-01-07,90,2001.19


store_id,store_region,store_size,open_date
1.0,West,2263.0,null
3.0,West,4989.0,2000-08-23
4.0,West,4942.0,2016-08-12
7.0,North,null,2005-07-19
8.0,West,691.0,null
11.0,East,null,2018-04-22
13.0,West,null,2009-10-03
14.0,North,1116.0,2003-06-30
18.0,West,1191.0,2016-01-23
19.0,North,null,2018-10-12


In [0]:
from pyspark.sql.functions import col, avg

store_df = store_df.filter(col("store_id").isNotNull())
#display(store_df.select(avg("store_size")).first()[0])
avg_store_size = store_df.select(avg("store_size")).first()[0]
store_df = store_df.fillna({"store_size": avg_store_size})
store_df = store_df.filter(col("open_date").isNotNull())
display(store_df.sample(fraction=0.5))


store_id,store_region,store_size,open_date
3.0,West,4989.0,2000-08-23
4.0,West,4942.0,2016-08-12
7.0,North,2606.2,2005-07-19
9.0,North,3653.0,2018-12-24
11.0,East,2606.2,2018-04-22
12.0,West,840.0,2000-03-22
13.0,West,2606.2,2009-10-03
16.0,East,2606.2,2003-10-01
17.0,East,2606.2,2010-09-23
18.0,West,1191.0,2016-01-23


In [0]:
from pyspark.sql.functions import col, to_date 

sales_df = sales_df.fillna({"quantity": 0, "total_amount": 0.0})
sales_df = sales_df.dropDuplicates()
sales_df = sales_df.filter(col("sale_id").isNotNull())
sales_df = sales_df.filter(col("store_id").isNotNull())
sales_df = sales_df.filter(col("sale_date").isNotNull())
sales_df = sales_df.filter((col("quantity") > 0) & (col("total_amount") > 0.0))

display(sales_df.sample(fraction=0.015))


sale_id,store_id,product_id,sale_date,quantity,total_amount
sale_560,15.0,4,2021-09-26,25,1177.75
sale_849,6.0,20,2020-03-15,9,306.65
sale_705,18.0,41,2022-12-06,65,1008.24
sale_366,14.0,33,2020-02-20,25,454.06
sale_246,8.0,9,2021-08-08,25,1051.71
sale_163,2.0,48,2022-04-28,47,950.63
sale_158,8.0,5,2021-03-16,86,1303.09
sale_462,7.0,11,2020-03-16,95,3746.55
sale_105,1.0,34,2022-07-04,42,211.15
sale_557,11.0,46,2020-10-16,53,1928.85


In [0]:
sales_store_df = sales_df.join(store_df, on="store_id", how="inner")
display(sales_store_df.sample(fraction=0.01))

store_id,sale_id,product_id,sale_date,quantity,total_amount,store_region,store_size,open_date
20.0,sale_139,46,2022-08-16,41,375.12,North,4867.0,2019-11-25
11.0,sale_559,1,2021-09-20,34,420.2,East,null,2018-04-22
16.0,sale_898,16,2020-06-28,3,139.42,East,null,2003-10-01
8.0,sale_909,3,2022-10-27,89,1274.02,West,691.0,null
13.0,sale_411,18,2020-07-03,77,1109.65,West,null,2009-10-03
19.0,sale_531,19,2020-07-03,36,356.3,North,null,2018-10-12
11.0,sale_592,21,2022-05-29,42,2007.06,East,null,2018-04-22


In [0]:
from pyspark.sql.functions import year, col, round

sales_store_df = sales_store_df.withColumn("sale_year", year(col("sale_date")))
sales_store_df = sales_store_df.filter(col("store_size").isNotNull())
sales_store_df = sales_store_df.withColumn("sales_per_sqft", round(col("total_amount") / col("store_size"), 2))
display(sales_store_df.sample(fraction=0.04))

store_id,sale_id,product_id,sale_date,quantity,total_amount,store_region,store_size,open_date,sale_year,sales_per_sqft
2.0,sale_691,30,2022-08-15,34,660.9,West,1510.0,2009-07-07,2022,0.44
9.0,sale_936,37,2020-11-22,68,1127.59,North,3653.0,2018-12-24,2020,0.31
2.0,sale_475,14,2022-12-04,31,211.15,West,1510.0,2009-07-07,2022,0.14
4.0,sale_350,4,2021-07-18,89,1784.07,West,4942.0,2016-08-12,2021,0.36
12.0,sale_728,42,2021-12-07,71,1721.84,West,840.0,2000-03-22,2021,2.05
3.0,sale_258,21,2021-09-29,92,4042.91,West,4989.0,2000-08-23,2021,0.81
14.0,sale_6,3,2022-08-20,83,522.16,North,1116.0,2003-06-30,2022,0.47
18.0,sale_998,36,2021-09-17,50,1167.35,West,1191.0,2016-01-23,2021,0.98


In [0]:
sales_store_df.createOrReplaceTempView("sales_store")

In [0]:
sales_store_sql = spark.sql("""
    select 
        store_id, 
        store_region,
        round(sum(total_amount),0) as total_sales,
        sum(quantity) as total_quantity
    from sales_store
    group by store_id, store_region
""")
display(sales_store_sql)

store_id,store_region,total_sales,total_quantity
20.0,North,53057.0,2138
1.0,West,41750.0,1556
8.0,West,67746.0,2404
3.0,West,40980.0,1638
4.0,West,38285.0,1240
2.0,West,47392.0,1648
12.0,West,34735.0,1372
14.0,North,41333.0,1778
9.0,North,49838.0,1902
18.0,West,32691.0,1503


In [0]:
top_product_sql = spark.sql("""
                                select 
                                    product_id,
                                    sum(quantity) as total_quantity
                                from sales_store
                                group by product_id
                                order by total_quantity desc
                                limit 5
                            """)
display(top_product_sql)

product_id,total_quantity
15,736
26,634
32,592
28,592
13,552


In [0]:
sales_store_sql.createOrReplaceTempView("store_sales_summary")
display(spark.sql("select * from store_sales_summary"))

store_id,store_region,total_sales,total_quantity
20.0,North,53057.0,2138
1.0,West,41750.0,1556
8.0,West,67746.0,2404
3.0,West,40980.0,1638
4.0,West,38285.0,1240
2.0,West,47392.0,1648
12.0,West,34735.0,1372
14.0,North,41333.0,1778
9.0,North,49838.0,1902
18.0,West,32691.0,1503


In [0]:
top_store_sql = spark.sql("""
                        select store_id,
                        total_sales
                        from store_sales_summary
                        order by total_sales desc
                        limit 5
                    """)
display(top_store_sql)                    

store_id,total_sales
8.0,67746.0
20.0,53057.0
9.0,49838.0
2.0,47392.0
1.0,41750.0


In [0]:
top_product_sql.write.parquet("/Volumes/upload/default/files/top_product")
top_store_sql.write.parquet("/Volumes/upload/default/files/top_store")